# Работа с базами данных в Python

В прошлый раз мы попробовали работать с базами через программу для компьютера, сегодня попробуем работать через питон. Также мы посмотрим подробнее на создание таблиц, добавление и удаление данных данных.

Для работы с базами SQLite есть специальная библиотека, она встроена в питон и не требует дополнительной установки.

In [32]:
import sqlite3
import pandas as pd

Взаимодействие с базой происходит с помощью двух компонентов - подключения и курсора. 

**Подключение** - это как бы тоннель, по которому передаются данные. 

**Курсор** - это инструмент, с помощью которого мы взаимодействуем с данными - с помощью курсора мы посылаем запросы и после выполнения курсор помнит, где мы остановились, если мы листаем результаты по частям. Курсор умеет делать только одно действие одновременно, поэтому нельзя посылать запрос, пока мы листаем результаты.

In [2]:
con = sqlite3.connect('imdb_small_indexed.db')  # подключение
cur = con.cursor()  # курсор

Попробуем для начала то, что уже умеем, то есть выберем данные по нашему условию. Курсор имеет метод execute, который выполняет нашу команду.

In [18]:
tom_hanks_query = """
SELECT title, premiered
FROM titles 
    JOIN crew ON titles.title_id = crew.title_id
    JOIN people ON crew.person_id = people.person_id
WHERE name = "Tom Hanks"
ORDER BY premiered DESC
LIMIT 5
"""

In [19]:
cur.execute(tom_hanks_query)

Вернулось что-то странное. Это потому что курсор не сразу возвращает данные, которые мы запросили. Нужно решить, как мы будем их читать.


**вариант 1**: все сразу

In [20]:
cur.fetchall()

[('Untitled Elvis Presley Project', 2021),
 ('BIOS', 2021),
 ('Untitled Elvis Presley Project', 2021),
 ('Borat Subsequent Moviefilm', 2020),
 ('Greyhound', 2020)]

Видим, что возвращается список кортежей, никаких красивых заголовков нет.

**вариант 2**: только 1

In [25]:
cur.execute(tom_hanks_query)
cur.fetchone()

('Untitled Elvis Presley Project', 2021)

дальше мы можем итерироваться и еще раз вызывать fetchone



In [26]:
while True:
    result = cur.fetchone()
    if result is not None:
        print(result)
    else:
        break

('BIOS', 2021)
('Untitled Elvis Presley Project', 2021)
('Borat Subsequent Moviefilm', 2020)
('Greyhound', 2020)


**вариант 3**: не по одному, а порциями

In [28]:
cur.execute(tom_hanks_query)
while True:
    result = cur.fetchmany(2)
    if len(result) > 0:
        print(result)
    else:
        break

[('Untitled Elvis Presley Project', 2021), ('BIOS', 2021)]
[('Untitled Elvis Presley Project', 2021), ('Borat Subsequent Moviefilm', 2020)]
[('Greyhound', 2020)]


### Задание 1

Выберите все фильмы после 2000, где играет ваша любимая актриса или актер. Считайте все данные (fetchall) и распечатайте.

1. запрос с соединением таблиц и условием по нужным столбцам
2. вывод

In [ ]:
# ваш код здесь

## Использование переменных в запросе

Допустим, что вы хотите сделать универсальную программу - она принимает на вход имя актера/актрисы и жанр, а потом выводит всю найденную информацию по такому запросу. Для того, чтобы создать переменные в запросе, мы должны оставить там знаки вопроса, туда подставится та информация, которую мы передадим потом в программе

In [29]:
name = "Natalie Portman"
genre = "Comedy"

In [30]:
act_genre_query = """
SELECT title, premiered
FROM titles 
    JOIN crew ON titles.title_id = crew.title_id
    JOIN people ON crew.person_id = people.person_id
    JOIN film_genres ON titles.title_id = film_genres.title_id
    JOIN genre_types ON film_genres.genre_id = genre_types.id
WHERE name = ? AND genre_name = ? AND premiered IS NOT NULL
ORDER BY premiered DESC
"""

In [31]:
cur.execute(act_genre_query, (name, genre))
cur.fetchall()

[('Your Highness', 2011),
 ('No Strings Attached', 2011),
 ('Hesher', 2010),
 ('New York, I Love You', 2008),
 ('Between Two Ferns with Zach Galifianakis', 2008),
 ("Mr. Magorium's Wonder Emporium", 2007),
 ('Free Zone', 2005),
 ('Garden State', 2004),
 ('Where the Heart Is', 2000),
 ('Anywhere But Here', 1999)]

### Задание 2

Попробуйте написать функцию, которая принимает на вход тип фильма (сериал, фильм), жанр, рейтинг и выводит фильмы, которые оценили больше 100 000 человек и рейтинг больше чем задано.

1. переменные
2. запрос с соединением таблиц и условием по нужным столбцам
3. вывод

In [ ]:
# ваш код здесь

Нельзя делать f-strings для запросов, есл вашей программой кто-то пользуется, кроме вас. Всегда, когда возможно, нужно использовать вопросы. Почему? На место переменной в f-string можно поместить запрос, который содержит вопрос на удаление или на получение доступа к информации в базе, которая не должна быть доступна.

### SQL и pandas

Можно читать результаты прямо в пандас с нужными названиями столбцов. Для этого есть метод read_sql_query. Туда мы передаем подключение и запрос.

In [34]:
df = pd.read_sql_query(act_genre_query, params = [name, genre], con=con)
df

,title,premiered
0,Your Highness,2011
1,No Strings Attached,2011
2,Hesher,2010
3,"New York, I Love You",2008
4,Between Two Ferns with Zach Galifianakis,2008
5,Mr. Magorium's Wonder Emporium,2007
6,Free Zone,2005
7,Garden State,2004
8,Where the Heart Is,2000
9,Anywhere But Here,1999


### Задание 4

Попробуйте повторить прошлое, чтобы данные были в виде датафрейма, выводятся title, rating, premiered

In [ ]:
# ваш код здесь

## Создание своих таблиц

Для создания таблицы нужно использовать ```CREATE```

Общий синтакис такой:

``` sql
CREATE TABLE table_name (column1 INT, column2 TEXT)
```

In [36]:
cur.execute("""
CREATE TABLE new_comedies (
    title_id INT, 
    title TEXT, 
    premiered INT,
    rating REAL,
    PRIMARY KEY (title_id)
)
""")

Ничего не сломалось, все работает. Теперь нужно **сохранить** то, что получилось. Для это нужно сделать коммит в подключении.

In [38]:
con.commit()

Чтобы отменить изменения, можно откатиться к созраненной версии (если вы еще не сделали коммит)

In [39]:
con.rollback()

Ок, мы создали таблицу, хотим туда добавить что-то. Сделаем выборку из общих данных и положим потом сюда, что получилось.

In [40]:
new_comedies_query = """
SELECT titles.title_id, title, premiered, rating
FROM titles
    JOIN film_genres ON film_genres.title_id = titles.title_id
    JOIN genre_types ON film_genres.genre_id = genre_types.id
    JOIN rating ON titles.title_id = rating.title_id
WHERE genre_types.genre_name = "Comedy" AND titles.premiered > 2019 AND titles.premiered < 2021
"""

In [41]:
cur.execute(new_comedies_query)
data = cur.fetchall()

data[:5]

[(446792, 'Surviving in L.A.', 2020, 8.1),
 (805647, 'The Witches', 2020, 5.2),
 (10004368, 'Jak najdalej stad', 2020, 7.2),
 (10006418, 'Avocado Toast the series', 2020, 7.3),
 (10023150, 'My Cousin', 2020, 5.8)]

Для того, чтобы что-то добавить в таблицу, мы используем ```INSERT```. У нас есть список однотипных кортежей, поэтому мы можем все их сразу добавить, а не по одному, для этого есть executemany.

In [42]:
cur.executemany("INSERT INTO new_comedies VALUES (?, ?, ?, ?)", data)
con.commit()

## Удаляем данные

Допустим, что мы хотим удалить все фильмы, которые начинаются на букву B. Посмотрим сначала, сколько таких.

Мы можем делать примерные запросы с помощью ```%```. Задать шаблон текста, типа поиск по подстроке.

In [45]:
cur.execute("SELECT COUNT(title_id) FROM new_comedies WHERE title LIKE 'B%'")
cur.fetchall()

[(73,)]

А теперь удалим все такие фильмы

In [47]:
cur.execute("DELETE FROM new_comedies WHERE title LIKE 'B%'")
con.commit()

Проверяем, что ничего не осталось.

In [48]:
cur.execute("SELECT COUNT(title_id) FROM new_comedies WHERE title LIKE 'B%'")
cur.fetchall()

[(0,)]

Чтобы удалить целую таблицу, мы можем ее дропнуть (```DROP```). Чтобы не было ошибки, если такой вообще не было, можно написать ```IF EXISTS```

In [50]:
cur.execute("DROP TABLE IF EXISTS new_comedies")
con.commit()

## Другие функции

На самом деле есть еще много интересных функций или параметров, например ```GROUP_CONCAT```, которая позволяет агрегировать текст, склеивая его через разделитель (как join в питоновских строчках). Или ```HAVING```, что позволяет фильтровать что-то уже после группировки, например, вы агрегируете данные и хотите вывести только те, где получилось больше 10 в этом параметре.


**Топ-10 фильмов по рейтингу, где ровно 2 жанра и 100 000 оценок**

Мы соединяем таблицы, фильтруем по числу оценок, потом группируем по айди фильма, потом посел группировки фильтруем по числу жанров, сортируем по убыванию рейтинга, а при равенстве по числу оценок, выбираем первые 10.

В агрегирующих функциях мы считаем число жанров для фильтра и склеиваем через запятую и пробел имена жанров.

In [61]:
cur.execute("""
SELECT title, premiered, COUNT(film_genres.genre_id) as n_genres, rating, votes, GROUP_CONCAT(genre_name, ', ')
FROM titles
    JOIN film_genres ON titles.title_id = film_genres.title_id
    JOIN genre_types ON film_genres.genre_id = genre_types.id
    JOIN rating ON titles.title_id = rating.title_id
WHERE rating.votes > 100000
GROUP BY titles.title_id
HAVING n_genres = 2
ORDER BY rating DESC, votes DESC
LIMIT 10
""")
cur.fetchall()

[('The Godfather', 1972, 2, 9.2, 1585284, 'Crime, Drama'),
 ('The Sopranos', 1999, 2, 9.2, 290309, 'Crime, Drama'),
 ('The Godfather: Part II', 1974, 2, 9.0, 1107605, 'Crime, Drama'),
 ('Pulp Fiction', 1994, 2, 8.9, 1793495, 'Crime, Drama'),
 ('Friends', 1994, 2, 8.9, 794054, 'Comedy, Romance'),
 ('12 Angry Men', 1957, 2, 8.9, 674893, 'Crime, Drama'),
 ('Forrest Gump', 1994, 2, 8.8, 1769908, 'Drama, Romance'),
 ('Peaky Blinders', 2013, 2, 8.8, 316333, 'Crime, Drama'),
 ('Freaks and Geeks', 1999, 2, 8.8, 126761, 'Comedy, Drama'),
 ('The Matrix', 1999, 2, 8.7, 1644392, 'Action, Sci-Fi')]

### Задание 5

Выберите несколько любимых актеров, попробуйте сделать таблицу, где будет информация о фильме: год, рейтинг, список жанров, список актеров из вашего списка.

Подсказка (запрос просто можно дописать)

``` sql
SELECT 
    ...
    COUNT(...) as n_my_actors,
    GROUP_CONCAT(...) as my_actors
FROM titles 
    ...
WHERE name IN (...)
GROUP BY titles.title_id
HAVING n_my_actors > 1
ORDER BY n_my_actors, premiered DESC
```

In [ ]:
# Ваш код здесь